In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
train_data = pd.read_csv('../input/train.csv')
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [3]:
def make_vocab(sentences):
    '''Build a vocabulary based on the available text data. Returns a dictionary with word and its frequency.'''
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [4]:
sentences = train_data['question_text'].apply(lambda x: x.split()).values
vocab = make_vocab(sentences)

In [5]:
import gensim
embeddings = gensim.models.KeyedVectors.load_word2vec_format('../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin',binary=True,limit=600000)

In [25]:
# Evaluating how word2vec works.
embeddings.most_similar('chelsea')
# Since this embedding was derived using Google news data, there's a large correlation between chelsea and wenger!

/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('liverpool', 0.8414958715438843),
 ('man_utd', 0.8178720474243164),
 ('wenger', 0.812255322933197),
 ('real_madrid', 0.7843406200408936),
 ('barca', 0.7839786410331726),
 ('utd', 0.7657824754714966),
 ('everton', 0.7584188580513),
 ('drogba', 0.7512555122375488),
 ('madrid', 0.7511358261108398),
 ('fergie', 0.7509583830833435)]

In [6]:
import operator
def present_in_word2vec(vocab,embeddings):
    '''Finds the intersection between embedding and our vocab.'''
    in_embedding = {}
    not_in_embedding = {}
    in_embedding_word_count = 0
    not_in_embedding_word_count = 0
    for word in vocab:
        try:
            in_embedding[word] = embeddings[word]
            in_embedding_word_count += vocab[word]
        except:
            not_in_embedding[word] = vocab[word]
            not_in_embedding_word_count += vocab[word]
            pass
    
    print('Found embeddings for {:.2%} of vocab'.format(len(in_embedding)/len(vocab)))
    print('Found embeddings for {:.2%} of all text'.format(in_embedding_word_count/(in_embedding_word_count+not_in_embedding_word_count)))
    x = sorted(not_in_embedding.items(),key=operator.itemgetter(1))[::-1]
    return x

In [7]:
out_of_embed = present_in_word2vec(vocab,embeddings) 

Found embeddings for 19.76% of vocab
Found embeddings for 78.24% of all text


In [8]:
out_of_embed[:10]

[('to', 403183),
 ('a', 402682),
 ('of', 330825),
 ('and', 251973),
 ('India?', 16384),
 ('it?', 12900),
 ("What's", 12425),
 ('do?', 8753),
 ('life?', 7753),
 ('you?', 6295)]

In [9]:
import string
p = string.punctuation
l = []
for y in p:
    if y not in ["'","-","/","&"]:
        l.append(y)
        
x_ = ''.join([x for x in l])
print(x_)
def clean_punctuation(token):
    '''Cleans text by adjusting punctuations'''   
    x = str(token)
    for punct in "-/":
        x = x.replace(punct,' ')
    for punct in '&':
        # & is present in the embedding hence adding spaces around it.
        x = x.replace(punct,f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct,'')
    
    return x
        

!"#$%()*+,.:;<=>?@[\]^_`{|}~


In [10]:
train_data['question_text'] = train_data['question_text'].apply(lambda x:clean_punctuation(x))

sentences = train_data['question_text'].apply(lambda x: x.split()).values
#print(sentences)
vocab = make_vocab(sentences)
out_of_embed = present_in_word2vec(vocab,embeddings)

Found embeddings for 44.37% of vocab
Found embeddings for 89.26% of all text


In [11]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [12]:
train_data["question_text"] = train_data["question_text"].apply(lambda x: clean_numbers(x))
sentences = train_data["question_text"].apply(lambda x: x.split())
vocab = make_vocab(sentences)

In [13]:
out_of_embed = present_in_word2vec(vocab,embeddings)

Found embeddings for 46.61% of vocab
Found embeddings for 90.02% of all text


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [15]:
# Preparing test data
test_data = pd.read_csv('../input/test.csv')
test_data['question_text'] = test_data['question_text'].fillna("_na_").values
test_data['question_text'] = test_data['question_text'].apply(lambda x:clean_punctuation(x))
test_data['question_text'] = test_data['question_text'].apply(lambda x:clean_numbers(x))
test_sent = test_data['question_text'].apply(lambda x:x.split())
test_vocab = make_vocab(test_sent)

In [16]:

from sklearn.model_selection import train_test_split
train_set,val_set = train_test_split(train_data,test_size=0.2)

max_features = 20000
maxlen = 80

train_X = train_set['question_text'].fillna("_na_").values
val_X = val_set['question_text'].fillna("_na_").values
test_X = test_data['question_text']

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

train_y = train_set['target'].values
val_y = val_set['target'].values

In [17]:
#tokenizer.word_index.items()
import gc

In [18]:
embeddings_index = {}
embedding_size = 300
for word in embeddings.wv.vocab:
    embeddings_index[word] = embeddings.word_vec(word)

all_embeddings = np.stack(list(embeddings_index.values()))
embed_mean,embed_std = all_embeddings.mean(),all_embeddings.std()
num_words = len(tokenizer.word_index)



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
embedding_matrix = np.random.normal(embed_mean,embed_std,(num_words,embedding_size))

for word,index in tokenizer.word_index.items():
    index -= 1
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

del(embeddings_index)
gc.collect()

0

In [20]:
from keras.layers import Dense,Input,Conv2D,Embedding,Dropout,Activation,MaxPool2D,Concatenate,Flatten,Reshape
from keras.models import Model
from keras.optimizers import Adam

inputs = Input(shape=(80,))
word2vec_embedding = Embedding(len(tokenizer.word_index),300,weights = [embedding_matrix],trainable=False)(inputs)
reshape = Reshape((80,300,1))(word2vec_embedding)

conv_window_3 = Conv2D(512,kernel_size=(3,300),padding='valid',activation='relu')(reshape)
conv_window_4 = Conv2D(512,kernel_size=(4,300),padding='valid',activation='relu')(reshape)
conv_window_5 = Conv2D(512,kernel_size=(5,300),padding='valid',activation='relu')(reshape)

max_pool_window_3 = MaxPool2D(pool_size=(80-3+1,1),strides=(1,1),padding='valid')(conv_window_3)
max_pool_window_4 = MaxPool2D(pool_size=(80-4+1,1),strides=(1,1),padding='valid')(conv_window_4)
max_pool_window_5 = MaxPool2D(pool_size=(80-5+1,1),strides=(1,1),padding='valid')(conv_window_5)

concat_feature_map = Concatenate(axis=1)([max_pool_window_3,max_pool_window_4,max_pool_window_5])
flatten = Flatten()(concat_feature_map)
dropout = Dropout(0.5)(flatten)

output = Dense(units=1,activation='sigmoid')(dropout)

model = Model(inputs=inputs,outputs=output)

adam = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_X,train_y, batch_size=30, epochs=2, verbose=1, validation_data=(val_X, val_y))


Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 1267s 1ms/step - loss: 0.1613 - acc: 0.9448 - val_loss: 0.1403 - val_acc: 0.9492
Epoch 2/2
1044897/1044897 [==============================] - 1263s 1ms/step - loss: 0.1442 - acc: 0.9495 - val_loss: 0.1344 - val_acc: 0.9509


In [21]:
from keras.models import load_model
model.save('QIQC_model.hd5')

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 80, 300)      53436900    input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 80, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 78, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [23]:
pred = model.predict(test_X)

In [24]:
#test_data.head()
sub = test_data[['qid']]
prediction = np.round(np.argmax(pred,axis=1)).astype(int)
sub['prediction'] = prediction
sub.to_csv('submission.csv',index=False)